In [1]:
import pandas as pd
import numpy as np
import time
from typing import List , Dict

In [2]:
df = pd.read_csv("data/yelp.csv")


In [3]:
#keeping only the column that we need that is the text as review and stars as the review rating
df = df[["text", "stars"]].dropna()

In [4]:
df = df.reset_index(drop=True)


In [5]:
df_small = df.sample(15, random_state=42).reset_index(drop=True)
len(df_small), df_small.head()


(15,
                                                 text  stars
 0  We got here around midnight last Friday... the...      4
 1  Brought a friend from Louisiana here.  She say...      5
 2  Every friday, my dad and I eat here. We order ...      3
 3  My husband and I were really, really disappoin...      1
 4  Love this place!  Was in phoenix 3 weeks for w...      5)

In [6]:
# Prompt V1: Simple JSON classification
prompt_v1 = """
You are an expert Yelp review analyst.

Task:
- Read the review text.
- Decide the most appropriate star rating from 1 to 5.

Return the result as a single JSON object with this exact structure:

{{
  "predicted_stars": <number from 1 to 5>,
  "explanation": "<very brief reason for the rating>"
}}

Respond with JSON only. No extra text.

Review:
"{review_text}"
"""

# Prompt V2: Step-by-step reasoning + JSON
prompt_v2 = """
You are an expert sentiment analyst for Yelp restaurant reviews.

Follow these steps in your head:
1. Identify key positive and negative points in the review.
2. Decide what star rating (1–5) a typical Yelp user would give.
3. Consider review length and intensity of sentiment.

Then respond ONLY with a single JSON object of this form:

{{
  "predicted_stars": <number from 1 to 5>,
  "explanation": "<1–2 short sentences summarizing your reasoning>"
}}

Do not add any text outside the JSON.

Review:
"{review_text}"
"""

# Prompt V3: Few-shot examples + JSON
prompt_v3 = """
You are an expert Yelp review rating assistant.

Below are examples of how you should think and respond.

Example 1
Review: "Terrible service. Food was cold and tasteless. I will never come back."
Response:
{{
  "predicted_stars": 1,
  "explanation": "Very negative sentiment about both service and food."
}}

Example 2
Review: "Pretty good burger and fries. Service was fine, nothing special."
Response:
{{
  "predicted_stars": 4,
  "explanation": "Overall positive with only minor issues."
}}

Now analyze the NEW review below and respond in the SAME JSON format:

{{
  "predicted_stars": <number from 1 to 5>,
  "explanation": "<brief explanation>"
}}

Review:
"{review_text}"
"""



In [1]:
import os
import google.generativeai as genai

api_key = os.getenv("GEMINI_API_KEY")
if api_key is None:
    raise ValueError("GEMINI_API_KEY environment variable not set.")
    
genai.configure(api_key=api_key)

C:\Users\asmit\anaconda3\envs\fynd\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [8]:
from google.generativeai import GenerativeModel
gemini_model = GenerativeModel("gemini-2.0-flash")

In [9]:
#now writing rating predictor function
import json
import re

def parse_model_output(output: str):
    """
    Try to extract a JSON object with `predicted_stars` and `explanation`
    from the model output.

    Returns:
        predicted (int), json_valid (bool)
    """

    # 1) Try direct JSON first
    try:
        data = json.loads(output)
        rating = int(data["predicted_stars"])
        if 1 <= rating <= 5 and "explanation" in data:
            return rating, True
    except Exception:
        pass

    # 2) Try to extract the first {...} block from the text
    try:
        match = re.search(r"\{.*\}", output, re.DOTALL)
        if match:
            json_str = match.group(0)
            # Try parsing that substring
            data = json.loads(json_str)
            rating = int(data["predicted_stars"])
            if 1 <= rating <= 5 and "explanation" in data:
                return rating, True
    except Exception:
        pass

    # 3) Last-resort: only extract rating digit from the text
    m = re.search(r"[1-5]", output)
    if m:
        rating = int(m.group())
    else:
        rating = 3  # neutral fallback

    return rating, False


def gemini_call(review_text: str, prompt_template: str):
    """
    Call Gemini with a given prompt template and review text.

    Returns:
        predicted_rating (int)
        json_valid (bool)
        raw_output (str)
    """
    prompt = prompt_template.format(review_text=review_text)

    try:
        response = gemini_model.generate_content(prompt)
        output = response.text.strip()

        predicted, json_valid = parse_model_output(output)
        return predicted, json_valid, output

    except Exception as e:
        print("Error calling Gemini:", e)
        # safe fallback
        return 3, False, ""



In [10]:
import time

def run_predictions(df, prompt_template, sleep_sec=4):
    preds = []
    json_flags = []
    raw_outputs = []

    total = len(df)

    for idx, row in df.iterrows():
        review = row["text"]
        rating, json_valid, raw = gemini_call(review, prompt_template)

        preds.append(rating)
        json_flags.append(json_valid)
        raw_outputs.append(raw)

        print(f"Processed {idx+1}/{total}")
        time.sleep(sleep_sec)   

    df_out = df.copy()
    df_out["pred_rating"] = preds
    df_out["json_valid"] = json_flags
    df_out["raw_output"] = raw_outputs
    return df_out


In [ ]:
df_v1 = run_predictions(df_small, prompt_v1)
df_v2 = run_predictions(df_small, prompt_v2)
df_v3 = run_predictions(df_small, prompt_v3)


Error calling Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]
Processed 1/15
Error calling Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]
Processed 2/15
Error calling Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]
Processed 3/15
Error calling Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googl

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate_prompt_results(df_results, name=""):
    y_true = df_results["stars"].astype(int)
    y_pred = df_results["pred_rating"].astype(int)

    acc = accuracy_score(y_true, y_pred)
    json_valid_rate = df_results["json_valid"].mean()

    print(f"-->{name}<--")
    print(f"Accuracy: {acc:.3f}")
    print(f"JSON validity rate: {json_valid_rate:.3f}")
    return {
        "prompt": name,
        "accuracy": acc,
        "json_valid_rate": json_valid_rate
    }


In [ ]:
metrics_v1 = evaluate_prompt_results(df_v1, "V1: Simple JSON")
metrics_v2 = evaluate_prompt_results(df_v2, "V2: Reasoning JSON")
metrics_v3 = evaluate_prompt_results(df_v3, "V3: Few-shot JSON")

In [ ]:
metrics_df = pd.DataFrame([metrics_v1, metrics_v2, metrics_v3])
metrics_df
